In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import math

import sklearn.metrics as sm

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, plot_confusion_matrix,accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor

from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

In [ ]:
df_org = pd.read_csv(r'BD Tratado\cluster.csv',sep=';')
df_org.info()

In [ ]:
df = df_org.drop(['MATRÍCULA', 'Mes', 'Mes_1', 'Mes_2', 'Mes_3', 'Mes_4', 'Mes_5', 'Mes_6', 'Mes_7', 'Mes_8', 'Mes_9', 'Mes_10', 'Mes_11', 'Mes_12', 
                          'SalarioMin','DiaSemana', 'DiaSemana_0', 'DiaSemana_1', 'DiaSemana_2', 'DiaSemana_3', 'DiaSemana_4', 'DiaSemana_5', 'DiaSemana_6',
                          'SO','SX','MANUAL','HORÁRIO INÍCIO','HORÁRIO FINAL'],axis=1)
df.info()

In [3]:
pipelines = []
pipelines.append(('LR', Pipeline([('Scaler', StandardScaler()),('LR', LinearRegression())])))
pipelines.append(('RD', Pipeline([('Scaler', StandardScaler()),('RD', Ridge())])))
pipelines.append(('LS', Pipeline([('Scaler', StandardScaler()),('LS', Lasso())])))
pipelines.append(('EL', Pipeline([('Scaler', StandardScaler()),('EL', ElasticNet())])))
pipelines.append(('KNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor(metric = 'manhattan', n_neighbors = 7, weights = 'distance'))])))
pipelines.append(('DTR', Pipeline([('Scaler', StandardScaler()),('DTR', DecisionTreeRegressor(max_leaf_nodes = 29, min_samples_split = 2))])))
pipelines.append(('RF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor())])))
pipelines.append(('ADA', Pipeline([('Scaler', StandardScaler()),('ADA', AdaBoostRegressor())])))

results = []
names = []


In [4]:
# Avaliação dos Pipelines
num_folds = 10
RMS = 'neg_mean_squared_error'

for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=25, shuffle = True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=RMS)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

NameError: name 'X_train' is not defined

In [ ]:
# Comparando os Algorítimos
fig = plt.figure(figsize=(20, 10))
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()
